In [1]:
import numpy as np
from scipy import ndimage
from scipy.signal import convolve2d
import skimage as sk
import skimage.io as skio
import skimage.transform as sktr
import matplotlib.pyplot as plt
import matplotlib
import cv2
import math

In [2]:
matplotlib.use('Agg')

# Part 1: Fun With Filters

## Part 1.1: Finite Difference Operator

In [52]:
cameraman = skio.imread("../media/cameraman.png", as_gray=True)
cameraman = sk.img_as_float(cameraman)

d_x, d_y = np.array([1, -1]).reshape((1, 2)), np.array([1, -1]).reshape((2, 1))

camera_x, camera_y = convolve2d(cameraman, d_x, mode='same', boundary='symm'), convolve2d(cameraman, d_y, mode='same', boundary='symm')

In [53]:
gradient_mag_img = np.sqrt(np.power(camera_x, 2) + np.power(camera_y, 2))

#### Binarize the gradient magnitude image

In [55]:
gradient_mag_copy = gradient_mag_img.flatten().copy()
mean, std = np.mean(gradient_mag_copy), np.std(gradient_mag_copy)

gradient_mag_copy = (gradient_mag_copy > (mean + (2.9 * std))).astype(np.float32)
gradient_mag_copy = gradient_mag_copy.reshape(542, 540)


In [56]:
fig, axes = plt.subplots(1, 4, figsize=(8, 6))

axes[0].imshow(camera_x, cmap="gray")
axes[0].axis('off')
axes[0].set_title("d/dx")

axes[1].imshow(camera_y, cmap="gray")
axes[1].axis('off')
axes[1].set_title("d/dy")

axes[2].imshow(gradient_mag_img, cmap="gray")
axes[2].axis('off')
axes[2].set_title("Gradient Magnitude Image")

axes[3].imshow(gradient_mag_copy, cmap="gray")
axes[3].axis('off')
axes[3].set_title("Gradient Magnitude Image Binarized")

plt.tight_layout()
plt.axis("off")
plt.show()

## Part 1.2: Derivative Theorem of Convolution

In [57]:
gaussian_oned = cv2.getGaussianKernel(ksize=5, sigma=1)
gaussian_twod = np.dot(gaussian_oned, gaussian_oned.T)

### Convolve the 2D Gaussian w/ image first, and then take the x & y derivatives of said image

In [58]:
blurred_cameraman = convolve2d(cameraman, gaussian_twod, mode='same', boundary='symm')

blurred_camera_x, blurred_camera_y = convolve2d(blurred_cameraman, d_x, mode='same', boundary='symm'), convolve2d(blurred_cameraman, d_y, mode='same', boundary='symm')
gradient_mag_img_blurred = np.sqrt(np.power(blurred_camera_x, 2) + np.power(blurred_camera_y, 2))

#### Take the x & y derivatives of the 2D Gaussian first and then convolve that with the image 

In [59]:
gaussian_der_x, gaussian_der_y = convolve2d(gaussian_twod, d_x, mode='same', boundary='symm'), convolve2d(gaussian_twod, d_y, mode='same', boundary='symm')

In [60]:
blurred_camera_x_2, blurred_camera_y_2 = convolve2d(cameraman, gaussian_der_x, mode='same', boundary='symm'), convolve2d(cameraman, gaussian_der_y, mode='same', boundary='symm')
gradient_mag_img_blurred_DoG = np.sqrt(np.power(blurred_camera_x_2, 2) + np.power(blurred_camera_y_2, 2))

In [61]:
fig, axes = plt.subplots(1, 2, figsize=(8, 6))

axes[0].imshow(gradient_mag_img_blurred, cmap="gray")
axes[0].axis('off')
axes[0].set_title("(d/dx)(h * f) + (d/dy)(h * f)")

axes[1].imshow(gradient_mag_img_blurred_DoG, cmap="gray")
axes[1].axis('off')
axes[1].set_title("((d/dx)(h) * f) + ((d/dy)(h) * f) -- DoG")

plt.tight_layout()
plt.axis("off")
plt.show()

# Part 2: Fun with Frequencies

## Part 2.1: Image "Sharpening"

### Taj Mahal Picture

In [39]:
taj = skio.imread("../media/taj.jpg")
taj = sk.img_as_float(taj)

plt.title("Regular Image")
plt.imshow(taj)
plt.show()

In [40]:
def manipulate_image(pic, gaussian, operation=None, alpha=2):
    new_img = pic.copy()

    for channel in range(3):
        color = pic[:, :, channel]
        blurred_color = convolve2d(color, gaussian, mode="same", boundary="symm")

        if operation == 'blur':
            new_img[:, :, channel] = blurred_color
        elif operation == 'sharpen':
            details = color - blurred_color
            new_img[:, :, channel] = color + alpha * details
    
    return new_img
    

In [42]:
blurred_taj = manipulate_image(taj, gaussian=gaussian_twod, operation="blur")

plt.title("Blurred Image")
plt.axis('off')
plt.imshow(blurred_taj)
plt.show()

In [43]:
sharpened_taj = manipulate_image(taj, gaussian=gaussian_twod, operation="sharpen", alpha=4)

plt.title("Sharpened Image, alpha=4")
plt.axis("off")
plt.imshow(sharpened_taj)
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


### Extra Images

In [48]:
porsche, mansion = skio.imread("../media/porsche.jpg"), skio.imread("../media/mansion.jpg")
porsche, mansion = sk.img_as_float(porsche), sk.img_as_float(mansion)

In [49]:
sharpened_porsche, sharpened_mansion = manipulate_image(porsche, gaussian=gaussian_twod, operation="sharpen", alpha=10), manipulate_image(mansion, gaussian=gaussian_twod, operation="sharpen", alpha=10)
plt.title("Sharpened Porsche Image")

fig, axes = plt.subplots(1, 4, figsize=(8, 6))

axes[0].imshow(porsche)
axes[0].axis('off')
axes[0].set_title("Regular Porsche")

axes[1].imshow(sharpened_porsche)
axes[1].axis('off')
axes[1].set_title("Sharpened Porsche, alpha=10")

axes[2].imshow(mansion)
axes[2].axis('off')
axes[2].set_title("Regular Mansion")

axes[3].imshow(sharpened_mansion)
axes[3].axis('off')
axes[3].set_title("Sharpened Mansion, alpha=10")

plt.tight_layout()
plt.axis("off")
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


## Part 2.2: Hybrid Images

### Align Images Starter Code

In [21]:
def get_points(im1, im2):
    original_backend = plt.get_backend()
    plt.switch_backend("qtagg")
    print('Please select 2 points in each image for alignment.')
    plt.imshow(im1)
    p1, p2 = plt.ginput(2)
    plt.close()
    plt.imshow(im2)
    p3, p4 = plt.ginput(2)
    plt.close()
    # plt.switch_backend(original_backend)
    return (p1, p2, p3, p4)

def recenter(im, r, c):
    R, C, _ = im.shape
    rpad = (int) (np.abs(2*r+1 - R))
    cpad = (int) (np.abs(2*c+1 - C))
    return np.pad(
        im, [(0 if r > (R-1)/2 else rpad, 0 if r < (R-1)/2 else rpad),
             (0 if c > (C-1)/2 else cpad, 0 if c < (C-1)/2 else cpad),
             (0, 0)], 'constant')

def find_centers(p1, p2):
    cx = np.round(np.mean([p1[0], p2[0]]))
    cy = np.round(np.mean([p1[1], p2[1]]))
    return cx, cy

def align_image_centers(im1, im2, pts):
    p1, p2, p3, p4 = pts
    h1, w1, b1 = im1.shape
    h2, w2, b2 = im2.shape
    
    cx1, cy1 = find_centers(p1, p2)
    cx2, cy2 = find_centers(p3, p4)

    im1 = recenter(im1, cy1, cx1)
    im2 = recenter(im2, cy2, cx2)
    return im1, im2

def rescale_images(im1, im2, pts):
    p1, p2, p3, p4 = pts
    len1 = np.sqrt((p2[1] - p1[1])**2 + (p2[0] - p1[0])**2) # distance between first pair of eyes
    len2 = np.sqrt((p4[1] - p3[1])**2 + (p4[0] - p3[0])**2) # distance between second pair of eyes
    dscale = len2/len1
    if dscale < 1: # distance between first pair is more
        im1 = sktr.rescale(im1, (dscale, dscale, 1)) 
    else: # distance between second pair is more
        im2 = sktr.rescale(im2, (1./dscale, 1./dscale, 1))
    return im1, im2

def rotate_im1(im1, im2, pts):
    p1, p2, p3, p4 = pts
    theta1 = math.atan2(-(p2[1] - p1[1]), (p2[0] - p1[0]))
    theta2 = math.atan2(-(p4[1] - p3[1]), (p4[0] - p3[0]))
    dtheta = theta2 - theta1
    im1 = sktr.rotate(im1, dtheta*180/np.pi)
    return im1, dtheta

def match_img_size(im1, im2):
    # Make images the same size
    h1, w1, c1 = im1.shape
    h2, w2, c2 = im2.shape
    if h1 < h2:
        im2 = im2[int(np.floor((h2-h1)/2.)) : -int(np.ceil((h2-h1)/2.)), :, :]
    elif h1 > h2:
        im1 = im1[int(np.floor((h1-h2)/2.)) : -int(np.ceil((h1-h2)/2.)), :, :]
    if w1 < w2:
        im2 = im2[:, int(np.floor((w2-w1)/2.)) : -int(np.ceil((w2-w1)/2.)), :]
    elif w1 > w2:
        im1 = im1[:, int(np.floor((w1-w2)/2.)) : -int(np.ceil((w1-w2)/2.)), :]
    assert im1.shape == im2.shape
    return im1, im2

def align_images(im1, im2):
    pts = get_points(im1, im2) # points that are selected
    im1, im2 = align_image_centers(im1, im2, pts)
    im1, im2 = rescale_images(im1, im2, pts)
    im1, angle = rotate_im1(im1, im2, pts)
    im1, im2 = match_img_size(im1, im2)
    return im1, im2

#### 2.2.1: Creating sets of Hybrid Pictures

In [86]:
image_sets = [
    ("../media/hybrid/DerekPicture.jpg", "../media/hybrid/nutmeg.jpg", 50, 20),
    ("../media/hybrid/bhatia.png", "../media/hybrid/praneet.png", 50, 20),
    ("../media/hybrid/deepika.jpeg", "../media/hybrid/jessica.jpg", 50, 20)
]

images_for_fft = {
    "Picture 1": None,
    "Picture 2": None,
    "Picture 1 Low-Frequency": None,
    "Picture 2 High-Frequency": None,
    "Hybrid Image": None
}

for i, tup in enumerate(image_sets):
    
    pic1, pic2 = skio.imread(tup[0]), skio.imread(tup[1])
    
    gaussian_hybrid = cv2.getGaussianKernel(ksize=tup[2], sigma=tup[3])

    pic1_aligned, pic2_aligned = align_images(pic1, pic2)

    pic1_blurred = manipulate_image(pic1_aligned, gaussian=gaussian_hybrid, operation="blur")
    pic2_blurred = manipulate_image(pic2_aligned, gaussian=gaussian_hybrid, operation="blur")

    pic2_high_freq = pic2_aligned - (pic2_blurred * 0.5)

    hybrid_image = np.clip(pic1_blurred + pic2_high_freq, 0, 255)

    if i == 1:
        images_for_fft["Picture 1"] = pic1
        images_for_fft["Picture 2"] = pic2
        images_for_fft["Picture 1 Low-Frequency"] = pic1_blurred
        images_for_fft["Picture 2 High-Frequency"] = pic2_high_freq
        images_for_fft["Hybrid Image"] = hybrid_image

    plt.imshow(hybrid_image)
    plt.show()

Please select 2 points in each image for alignment.


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Please select 2 points in each image for alignment.


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Please select 2 points in each image for alignment.


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


#### 2.2.2: Frequency Analysis of Hybrid Images and their Inputs

In [87]:
fig, axes = plt.subplots(1, 5, figsize=(8, 6))
i = 0

for k, img in images_for_fft.items():

    gray_image = cv2.cvtColor(img.astype(np.float32), cv2.COLOR_RGB2GRAY)

    f_transform = np.fft.fft2(gray_image)
    f_shift = np.fft.fftshift(f_transform)

    magnitude_spectrum = np.abs(f_shift)

    log_magnitude = np.log(1 + magnitude_spectrum)

    axes[i].imshow(log_magnitude)
    axes[i].axis('off')
    axes[i].set_title(k)

    i += 1

plt.tight_layout()
plt.show()


[[[ 62  57  43 255]
  [ 62  57  41 255]
  [ 69  63  47 255]
  ...
  [147 138 117 255]
  [147 137 116 255]
  [146 137 112 255]]

 [[ 62  57  43 255]
  [ 63  59  42 255]
  [ 68  64  45 255]
  ...
  [148 138 116 255]
  [148 138 117 255]
  [145 135 112 255]]

 [[ 61  56  42 255]
  [ 64  60  42 255]
  [ 69  65  46 255]
  ...
  [146 138 116 255]
  [149 139 118 255]
  [143 135 113 255]]

 ...

 [[ 59  53  38 255]
  [ 63  56  42 255]
  [ 65  57  43 255]
  ...
  [ 30  36  37 255]
  [ 23  29  30 255]
  [ 22  27  29 255]]

 [[ 59  53  39 255]
  [ 65  57  43 255]
  [ 65  58  43 255]
  ...
  [ 31  37  38 255]
  [ 20  25  27 255]
  [ 20  25  26 255]]

 [[ 55  49  36 255]
  [ 63  56  42 255]
  [ 65  56  41 255]
  ...
  [ 19  25  24 255]
  [ 19  24  25 255]
  [ 26  30  31 255]]]
[[[ 12  13  10 255]
  [ 13  14  10 255]
  [ 14  14  10 255]
  ...
  [  0   0   0 255]
  [  0   0   0 255]
  [  0   0   0 255]]

 [[ 13  15  10 255]
  [ 14  15  11 255]
  [ 14  16  11 255]
  ...
  [  0   0   0 255]
  [  0   0  

## Part 2.3: Gaussian and Laplacian Stacks

In [65]:
apple, orange = skio.imread("../media/blending/apple.jpeg").astype(np.float32), skio.imread("../media/blending/orange.jpeg").astype(np.float32)

In [93]:
def blend_two_images(pic1, pic2, ksize=25, sigma=15, mask=np.tile(np.linspace(1, 0, 300), (300, 1))):
    imA_gaussian_stack, imB_gaussian_stack = [pic1], [pic2]
    gaussian_for_blurring = cv2.getGaussianKernel(ksize=ksize, sigma=sigma)

    gaussian_for_blurring = np.dot(gaussian_for_blurring, gaussian_for_blurring.T)

    for i in range(5): # number of iterations -- change as desired
        imA_gaussian_stack.append(manipulate_image(imA_gaussian_stack[-1], operation="blur", gaussian=gaussian_for_blurring))
        imB_gaussian_stack.append(manipulate_image(imB_gaussian_stack[-1], operation="blur", gaussian=gaussian_for_blurring))

    laplacian_A, laplacian_B = [], []

    not_normalized_laplacian_A, not_normalized_laplacian_B = [], []

    mask_gaussian = [mask]

    for k in range(len(imA_gaussian_stack)):
        if k == len(imA_gaussian_stack) - 1:
            break
        
        # normalize

        difference_A = imA_gaussian_stack[k] - imA_gaussian_stack[k + 1]
        difference_B = imB_gaussian_stack[k] - imB_gaussian_stack[k + 1]
        
        normalized_difference_A = difference_A - np.min(difference_A)
        normalized_difference_A = normalized_difference_A / np.max(normalized_difference_A)

        normalized_difference_B = difference_B - np.min(difference_B)
        normalized_difference_B = normalized_difference_B / np.max(normalized_difference_B)

        laplacian_A.append(normalized_difference_A)
        laplacian_B.append(normalized_difference_B)

        not_normalized_laplacian_A.append(difference_A)
        not_normalized_laplacian_B.append(difference_B)

        # run gaussian blur on mask

        mask_gaussian.append(convolve2d(mask_gaussian[-1], gaussian_for_blurring, mode="same", boundary="symm"))


    laplacian_blend = []

    for A, B, C in zip(not_normalized_laplacian_A, not_normalized_laplacian_B, mask_gaussian):

        blend = np.zeros_like(not_normalized_laplacian_A[0])

        for channel in range(3):
            color_imA, color_imB = A[:, :, channel], B[:, :, channel]

            blend[:, :, channel] = (C * color_imA) + ((1 - C) * color_imB)

        laplacian_blend.append(blend)

    # handle extra level of mask blur stack

    coarsest_level = np.zeros_like(imA_gaussian_stack[-1])

    last_gaussian_A, last_gaussian_B = imA_gaussian_stack[-1], imB_gaussian_stack[-1]

    for channel in range(3):
        color_imA, color_imB = last_gaussian_A[:, :, channel], last_gaussian_B[:, :, channel]
        coarsest_level[:, :, channel] = (mask_gaussian[-1] * color_imA) + ((1 - mask_gaussian[-1]) * color_imB)

    laplacian_blend.append(coarsest_level)

    final_blend = np.zeros_like(laplacian_blend[0])

    for i in range(len(laplacian_blend)):
        final_blend += laplacian_blend[i]

    final_blend = final_blend - np.min(final_blend)
    final_blend = final_blend / np.max(final_blend)

    return (laplacian_A, laplacian_B, final_blend)

In [95]:
regular_mask = np.tile(np.linspace(1, 0, 300), (300, 1))
binary_mask = np.where(regular_mask < 0.5, 0, 1)

apple_laplacian, orange_laplacian, apple_orange_blended = blend_two_images(apple, orange, mask=binary_mask)

plt.imshow(apple_orange_blended)
plt.axis("off")
plt.show()

In [92]:
bhatia, praneet = skio.imread("../media/hybrid/bhatia.jpg"), skio.imread("../media/hybrid/praneet.jpg")
bhatia, praneet = sk.transform.resize(bhatia, (300, 300), anti_aliasing=True), sk.transform.resize(praneet, (300, 300), anti_aliasing=True)

In [96]:
bhatia_laplacian, praneet_laplacian, bhatia_praneet_blended = blend_two_images(bhatia, praneet, ksize=50, sigma=20, mask=binary_mask)

plt.imshow(bhatia_praneet_blended)
plt.axis("off")
plt.show()

In [90]:
height, width, radius = 300, 300, 100
y, x = np.ogrid[:height, :width]

center_x, center_y = width // 2, height // 2
distance_from_center = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)

circle_mask = np.where(distance_from_center >= radius, 0, 1)

In [97]:
alex, shadt = skio.imread("../media/blending/alex.jpg"), skio.imread("../media/blending/shadt.jpg")
alex, shadt = sk.transform.resize(alex, (300, 300), anti_aliasing=True), sk.transform.resize(shadt, (300, 300), anti_aliasing=True)

In [101]:
alex_laplacian, shadt_laplacian, alex_shadt_blended = blend_two_images(alex, shadt, ksize=30, sigma=20, mask=circle_mask)

plt.imshow(alex_shadt_blended)
plt.axis("off")
plt.show()

: 

#### Display Laplacian Stacks

In [199]:
fig, axes = plt. subplots(1, 5, figsize=(8, 6))

for i, laplacian in enumerate(apple_laplacian):
    axes[i].imshow(laplacian)
    axes[i].axis('off')
    axes[i].set_title(f"Laplacian #{i}")

plt.tight_layout()
plt.show()

In [200]:
fig, axes = plt. subplots(1, 5, figsize=(8, 6))

for i, laplacian in enumerate(orange_laplacian):
    axes[i].imshow(laplacian)
    axes[i].axis('off')
    axes[i].set_title(f"Laplacian #{i}")

plt.tight_layout()
plt.show()